# Feature Engineering with Pandas

This notebook covers various aspects of handling data in Pandas, including types of data, selecting data types, dealing with categorical variables, encoding, handling uncommon categories, numeric variables, binarizing, binning, and handling missing data.

We will use real datasets:
- Stack Overflow Developer Survey 2023: https://raw.githubusercontent.com/Stephen137/stack_overflow_developer_survey_2023/main/data/survey_results_public_2023.csv
- NYC Restaurant Inspection Results: https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD

Note: The Stack Overflow data has columns like 'Country', 'ConvertedCompYearly' (similar to ConvertedSalary), etc.

## Types of Data

- Continuous data
- Categorical (e.g., gender, birth country)
- Ordinal: order without actual distance
- Boolean
- Date time

## Exercise 1: Loading Data and Checking Types

In [3]:
import pandas as pd

# Define the URL for Stack Overflow survey
so_survey_csv = "survey_results_schema.csv"
# Load the data
so_survey_df = pd.read_csv(so_survey_csv)

# Print the first five rows
print(so_survey_df.head())

# Print the data types
print('\nColumn Data Types:')
print(so_survey_df.dtypes)

      qid       qname                                           question  \
0    QID2  MainBranch  Which of the following options best describes ...   
1  QID127         Age                                 What is your age?*   
2  QID296  Employment  Which of the following best describes your cur...   
3  QID308  RemoteWork  Which best describes your current work situation?   
4  QID341       Check  Just checking to make sure you are paying atte...   

  force_resp type selector  
0       True   MC     SAVR  
1       True   MC     SAVR  
2       True   MC     MAVR  
3      False   MC     SAVR  
4       True   MC     SAVR  

Column Data Types:
qid           object
qname         object
question      object
force_resp    object
type          object
selector      object
dtype: object


## Selecting Specific Data Types

In [4]:
# Create subset of only the numeric columns
so_numeric_df = so_survey_df.select_dtypes(include=['int', 'float'])

# Print the column names
print(so_numeric_df.columns)

Index([], dtype='object')


## Dealing with Categorical Variables

We encode categorical variables into numbers or booleans.

Types of encoding:
1. One-Hot Encoding: n categories into n features.
2. Dummy Encoding: n categories into n-1 features, omitting one to avoid collinearity.

## One-Hot Encoding Example

In [8]:
import pandas as pd

# Create a sample dataset
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank', 'Grace', 'Hannah', 'Ian', 'Jack',
             'Kira', 'Liam', 'Mona', 'Nina', 'Oscar', 'Paul', 'Quinn', 'Rita', 'Sam', 'Tina'],
    'Country': ['USA', 'India', 'USA', 'Germany', 'India', 'Nepal', 'Germany', 'USA', 'Nepal', 'India',
                'Germany', 'USA', 'Nepal', 'India', 'USA', 'Germany', 'Nepal', 'India', 'USA', 'Nepal']
}

df = pd.DataFrame(data)

# Display the dataset
# print(df)


# Convert the Country column to one-hot encoded DataFrame
one_hot_encoded = pd.get_dummies(df, columns=['Country'], prefix='OH')

# Print the column names
print(one_hot_encoded.columns)

Index(['Name', 'OH_Germany', 'OH_India', 'OH_Nepal', 'OH_USA'], dtype='object')


## Dummy Encoding Example

In [10]:
# Create dummy variables for the Country column
dummy = pd.get_dummies(df, columns=['Country'], drop_first=True, prefix='DM')

# Print the column names
print(dummy.columns)

Index(['Name', 'DM_India', 'DM_Nepal', 'DM_USA'], dtype='object')


## Dealing with Uncommon Categories

In [13]:
# Create a series out of the Country column
countries = df['Country']

# Get the counts of each category
country_counts = countries.value_counts()

# Create a mask for categories that occur less than 10 times
mask = countries.isin(country_counts[country_counts < 10].index)

# Label all other categories as 'Other'
df.loc[mask, 'Country'] = 'Other'

# Print the updated category counts
print(df['Country'].value_counts())

Country
Other    20
Name: count, dtype: int64


## Numeric Variables

Example with Restaurant Data: Binarizing violations.

In [17]:
# Load NYC Restaurant Inspection Data
restaurant_csv = 'https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD'
restaurant_df = pd.read_csv(restaurant_csv)

# Print head
print(restaurant_df.head())

# For simplicity, assume 'SCORE' represents violation score (higher score = more violations)
# Create a binary column: has_violation if SCORE > 0
restaurant_df['has_violation'] = 0
restaurant_df.loc[restaurant_df['SCORE'] > 0, 'has_violation'] = 1

# Print sample
print(restaurant_df[['SCORE', 'has_violation']].head())

      CAMIS                  DBA       BORO BUILDING             STREET  \
0  50175073      KUCHELA KUIZINE   Brooklyn     1197    FLATBUSH AVENUE   
1  50170738  787 COFFEE WEST LLC  Manhattan      245   WEST   46 STREET   
2  50036660          KIKOO SUSHI  Manhattan      141           1 AVENUE   
3  50172620           FRIJOLEROS   Brooklyn      131  GREENPOINT AVENUE   
4  50171814     ACE SHAWARMA INC      Bronx     3455      JEROME AVENUE   

   ZIPCODE       PHONE CUISINE DESCRIPTION INSPECTION DATE  \
0  11226.0  3473355072                 NaN      01/01/1900   
1  10036.0  9082308846                 NaN      01/01/1900   
2  10003.0  2125333888            Japanese      03/25/2024   
3  11222.0  3473842957                 NaN      01/01/1900   
4  10467.0  6467023905                 NaN      01/01/1900   

                                            ACTION  ...  \
0                                              NaN  ...   
1                                              NaN  ...   

## Binning Numeric Data

In [ ]:
# Back to SO data for salary binning
# Note: Column is 'ConvertedCompYearly'

# Create Paid_Job column filled with zeros
so_survey_df['Paid_Job'] = 0

# Replace where ConvertedCompYearly > 0
so_survey_df.loc[so_survey_df['ConvertedCompYearly'] > 0, 'Paid_Job'] = 1

# Print sample
print(so_survey_df[['Paid_Job', 'ConvertedCompYearly']].head())

In [ ]:
import numpy as np

# Specify bin boundaries
bins = [-np.inf, 10000, 50000, 100000, 150000, np.inf]

# Bin labels
labels = ['Very low', 'Low', 'Medium', 'High', 'Very high']

# Bin the ConvertedCompYearly
so_survey_df['boundary_binned'] = pd.cut(so_survey_df['ConvertedCompYearly'], bins=bins, labels=labels)

# Print sample
print(so_survey_df[['boundary_binned', 'ConvertedCompYearly']].head())

# Day 2
## Handling Gaps in Data (Missing Values)

In [ ]:
# Check info
so_survey_df.info()

# Check missing values
print(so_survey_df.isnull().sum())

In [ ]:
# For restaurant data
restaurant_df.info()

print(restaurant_df.isnull().sum())

In [ ]:
# Check data info and null values
print(so_survey_df.info())
print(so_survey_df.isnull().sum())

In [ ]:
# Subset the DataFrame and print the number of non-missing values
sub_df = so_survey_df[['Age', 'Gender', 'ConvertedSalary']]
print(sub_df.count())

In [ ]:
# Print the top 10 entries of the DataFrame
print(sub_df.head(10))

In [ ]:
# Print the locations of the missing values
print(sub_df.head(10).isnull())

In [ ]:
# Print the locations of the non-missing values
print(sub_df.head(10).notnull())

In [ ]:
# Print the number of rows and columns
print(so_survey_df.shape)

In [ ]:
# Create a new DataFrame dropping all incomplete rows
no_missing_values_rows = so_survey_df.dropna(how='any')
print(no_missing_values_rows.shape)

In [ ]:
# Create a new DataFrame dropping all columns with incomplete rows
no_missing_values_cols = so_survey_df.dropna(how='any', axis=1)
print(no_missing_values_cols.shape)

In [ ]:
# Drop all rows where Gender is missing
no_gender = so_survey_df.dropna(subset=['Gender'])
print(no_gender.shape)

In [ ]:
# Replace missing values in Gender with 'Not Given'
so_survey_df['Gender'].fillna(value='Not Given', inplace=True)
print(so_survey_df['Gender'].value_counts())

In [ ]:
# Print the first five rows of StackOverflowJobsRecommend column
print(so_survey_df['StackOverflowJobsRecommend'].head())

In [ ]:
# Fill missing values with the mean
so_survey_df['StackOverflowJobsRecommend'].fillna(so_survey_df['StackOverflowJobsRecommend'].mean(), inplace=True)
print(so_survey_df['StackOverflowJobsRecommend'].head())

In [ ]:
# Fill missing values with the mean and round
so_survey_df['StackOverflowJobsRecommend'].fillna(so_survey_df['StackOverflowJobsRecommend'].mean(), inplace=True)
so_survey_df['StackOverflowJobsRecommend'] = round(so_survey_df['StackOverflowJobsRecommend'])
print(so_survey_df['StackOverflowJobsRecommend'].head())

In [ ]:
# Remove commas in the RawSalary column
so_survey_df['RawSalary'] = so_survey_df['RawSalary'].str.replace(',', '')

In [ ]:
# Remove dollar signs in the RawSalary column
so_survey_df['RawSalary'] = so_survey_df['RawSalary'].str.replace('$', '')

In [ ]:
# Attempt to convert RawSalary to numeric, coercing errors to NaN
numeric_vals = pd.to_numeric(so_survey_df['RawSalary'], errors='coerce')
idx = numeric_vals.isna()
print(so_survey_df['RawSalary'][idx])

In [ ]:
# Replace pound signs and convert RawSalary to float
so_survey_df['RawSalary'] = so_survey_df['RawSalary'].str.replace('£', '')
so_survey_df['RawSalary'] = so_survey_df['RawSalary'].astype('float')
print(so_survey_df['RawSalary'])

In [ ]:
# Use method chaining to clean and convert RawSalary
so_survey_df['RawSalary'] = so_survey_df['RawSalary']\
    .str.replace(',', '')\
    .str.replace('$', '')\
    .str.replace('£', '')\
    .astype('float')
print(so_survey_df['RawSalary'])

# day 3

In [ ]:
# Import necessary libraries for visualization and preprocessing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
# Create a histogram of all numeric columns
# Visualizes the distribution of numerical features
so_numeric_df.hist()
plt.show()

In [ ]:
# Create a boxplot for Age and Years Experience
# Helps identify the spread and potential outliers in these columns
so_numeric_df[['Age', 'Years Experience']].boxplot()
plt.show()

In [ ]:
# Plot pairwise relationships for all numeric columns
# Uses seaborn to show scatter plots and histograms for feature interactions
sns.pairplot(so_numeric_df)
plt.show()

In [ ]:
# Plot pairwise relationships for all numeric columns
# Uses seaborn to show scatter plots and histograms for feature interactions
sns.pairplot(so_numeric_df)
plt.show()

In [ ]:
# Apply MinMaxScaler to scale Age between 0 and 1
# Ensures features are on the same scale for models sensitive to magnitude
MM_scaler = MinMaxScaler()
MM_scaler.fit(so_numeric_df[['Age']])
so_numeric_df['Age_MM'] = MM_scaler.transform(so_numeric_df[['Age']])
print(so_numeric_df[['Age_MM', 'Age']].head())

In [ ]:
# Apply StandardScaler to standardize Age
# Centers the data around mean with unit standard deviation
SS_scaler = StandardScaler()
SS_scaler.fit(so_numeric_df[['Age']])
so_numeric_df['Age_SS'] = SS_scaler.transform(so_numeric_df[['Age']])
print(so_numeric_df[['Age_SS', 'Age']].head())

In [ ]:
# Apply PowerTransformer to reduce skewness in ConvertedSalary
# Uses Box-Cox or Yeo-Johnson to make data more normally distributed
pow_trans = PowerTransformer()
pow_trans.fit(so_numeric_df[['ConvertedSalary']])
so_numeric_df['ConvertedSalary_LG'] = pow_trans.transform(so_numeric_df[['ConvertedSalary']])
so_numeric_df[['ConvertedSalary', 'ConvertedSalary_LG']].hist()
plt.show()

In [ ]:
# Apply PowerTransformer to reduce skewness in ConvertedSalary
# Uses Box-Cox or Yeo-Johnson to make data more normally distributed
pow_trans = PowerTransformer()
pow_trans.fit(so_numeric_df[['ConvertedSalary']])
so_numeric_df['ConvertedSalary_LG'] = pow_trans.transform(so_numeric_df[['ConvertedSalary']])
so_numeric_df[['ConvertedSalary', 'ConvertedSalary_LG']].hist()
plt.show()

In [ ]:
# Remove outliers using the 95th quantile for ConvertedSalary
# Trims the top 5% of data to reduce the impact of extreme values
quantile = so_numeric_df['ConvertedSalary'].quantile(0.95)
trimmed_df = so_numeric_df[so_numeric_df['ConvertedSalary'] < quantile]
so_numeric_df[['ConvertedSalary']].hist()
plt.show()
plt.clf()
trimmed_df[['ConvertedSalary']].hist()
plt.show()

In [ ]:
# Remove outliers using statistical method (3 standard deviations)
# Trims data points outside 3 standard deviations from the mean
std = so_numeric_df['ConvertedSalary'].std()
mean = so_numeric_df['ConvertedSalary'].mean()
cut_off = std * 3
lower, upper = mean - cut_off, mean + cut_off
trimmed_df = so_numeric_df[(so_numeric_df['ConvertedSalary'] < upper) & (so_numeric_df['ConvertedSalary'] > lower)]
trimmed_df[['ConvertedSalary']].boxplot()
plt.show()

In [ ]:
# Apply StandardScaler to training data and transform test data
# Ensures no data leakage by fitting scaler only on training data
SS_scaler = StandardScaler()
SS_scaler.fit(so_train_numeric[['Age']])
so_test_numeric['Age_ss'] = SS_scaler.transform(so_test_numeric[['Age']])
print(so_test_numeric[['Age', 'Age_ss']].head())

In [ ]:
# Remove outliers in test data using training data thresholds
# Uses mean and standard deviation from training data to avoid leakage
train_std = so_train_numeric['ConvertedSalary'].std()
train_mean = so_train_numeric['ConvertedSalary'].mean()
cut_off = train_std * 3
train_lower, train_upper = train_mean - cut_off, train_mean + cut_off
trimmed_df = so_test_numeric[(so_test_numeric['ConvertedSalary'] < train_upper) & (so_test_numeric['ConvertedSalary'] > train_lower)]

In [ ]:
# Remove outliers in test data using training data thresholds
# Uses mean and standard deviation from training data to avoid leakage
train_std = so_train_numeric['ConvertedSalary'].std()
train_mean = so_train_numeric['ConvertedSalary'].mean()
cut_off = train_std * 3
train_lower, train_upper = train_mean - cut_off, train_mean + cut_off
trimmed_df = so_test_numeric[(so_test_numeric['ConvertedSalary'] < train_upper) & (so_test_numeric['ConvertedSalary'] > train_lower)]

In [ ]:
# Print the first 5 rows of the text column
# Displays raw text data for inspection
print(speech_df['text'].head(5))

In [ ]:
# Clean text by removing non-letters and converting to lowercase
# Prepares text for further processing by standardizing it
speech_df['text_clean'] = speech_df['text'].str.replace('[^a-zA-Z]', ' ')
speech_df['text_clean'] = speech_df['text_clean'].str.lower()
print(speech_df['text_clean'].head())

In [ ]:
# Extract high-level text features
# Calculates character count, word count, and average word length
speech_df['char_cnt'] = speech_df['text_clean'].str.len()
speech_df['word_cnt'] = speech_df['text_clean'].str.split().str.len()
speech_df['avg_word_length'] = speech_df['char_cnt'] / speech_df['word_cnt']
print(speech_df[['text_clean', 'char_cnt', 'word_cnt', 'avg_word_length']])

In [ ]:
# Count words using CountVectorizer
# Creates a sparse matrix of word counts
cv = CountVectorizer()
cv.fit(speech_df['text_clean'])
print(cv.get_feature_names_out())

In [ ]:
# Transform text to word count array
# Converts text to a matrix of word frequencies
cv_transformed = cv.transform(speech_df['text_clean'])
cv_array = cv_transformed.toarray()
print(cv_array)
print(cv_array.shape)

In [ ]:
# Limit features with min_df and max_df
# Reduces features by keeping words in 20-80% of documents
cv = CountVectorizer(min_df=0.2, max_df=0.8)
cv_transformed = cv.fit_transform(speech_df['text_clean'])
cv_array = cv_transformed.toarray()
print(cv_array.shape)

In [ ]:
# Create DataFrame from word counts
# Adds word count features to the original DataFrame
cv_df = pd.DataFrame(cv_array, columns=cv.get_feature_names_out()).add_prefix('Counts_')
speech_df_new = pd.concat([speech_df, cv_df], axis=1, sort=False)
print(speech_df_new.head())

In [ ]:
# Apply TfidfVectorizer with feature limits and stop words
# Creates TF-IDF features, reducing the impact of common words
tv = TfidfVectorizer(max_features=100, stop_words='english')
tv_transformed = tv.fit_transform(speech_df['text_clean'])
tv_df = pd.DataFrame(tv_transformed.toarray(), columns=tv.get_feature_names_out()).add_prefix('TFIDF_')
print(tv_df.head())

In [ ]:
# Inspect top 5 TF-IDF words in the first document
# Identifies the most important words in a single document
sample_row = tv_df.iloc[0]
print(sample_row.sort_values(ascending=False).head(5))

In [ ]:
# Transform test data using fitted TfidfVectorizer
# Applies the same transformation to avoid data leakage
tv = TfidfVectorizer(max_features=100, stop_words='english')
tv_transformed = tv.fit_transform(train_speech_df['text_clean'])
test_tv_transformed = tv.transform(test_speech_df['text_clean'])
test_tv_df = pd.DataFrame(test_tv_transformed.toarray(), columns=tv.get_feature_names_out()).add_prefix('TFIDF_')
print(test_tv_df.head())

In [ ]:
# Create trigram features using CountVectorizer
# Extracts three-word phrases to capture context
cv_trigram_vec = CountVectorizer(max_features=100, stop_words='english', ngram_range=(3, 3))
cv_trigram = cv_trigram_vec.fit_transform(speech_df['text_clean'])
print(cv_trigram_vec.get_feature_names_out())

In [ ]:
# Find the most common trigrams
# Identifies frequently occurring three-word phrases
cv_tri_df = pd.DataFrame(cv_trigram.toarray(), columns=cv_trigram_vec.get_feature_names_out()).add_prefix('Counts_')
print(cv_tri_df.sum().sort_values(ascending=False).head())

# THE END 
## TO BE CONTINUED